In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
print("INIT module_path: ", module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

DATA_DIR = module_path + "/data"
MODEL_DIR = module_path + "/model"

for data_dir in [DATA_DIR, MODEL_DIR]:
    os.makedirs(data_dir, exist_ok=True)

INIT module_path:  c:\Users\Alan\Desktop\Open_Source\BERT-TLSA-paper


In [2]:
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding
from datasets import load_dataset

In [ ]:
dataset_dict = load_dataset("shawhin/phishing-site-classification") 

In [ ]:
model_path = "google-bert/bert-base-uncased"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load model with binary classification head
id2label = { 0: "Safe", 1: "Not Safe" }
label2id = { "Sade": 0, "Not Safe": 1 }
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2, id2label=id2label, label2id=label2id)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset_dict.get("train")[:5]

{'text': ["http://bazurashop.com/idex.html?sfm_from_iframe=1',300,350",
  'hollywoodland.org/?p=29',
  'tunnekylmyysmiddletonii.02leds.com/me4xcdste0.php\\n',
  'usa-people-search.com/Find-Carla-Brown-IA.aspx',
  'inspire-consultants.com.my/487ygfh'],
 'labels': [1, 0, 1, 0, 1]}

In [ ]:
# Freeze parameters except for pooling layer
for name, param in model.named_parameters():
    param.requires_grad = "pooler" in name

In [ ]:
def preprocess_fn(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_data = dataset_dict.map(preprocess_fn, batched=True)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [ ]:
# Feeds data for training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)

    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)["roc_auc"], 3)

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)["accuracy"], 3)

    return {"Accuracy": acc, "AUC": auc}

In [ ]:
# hyperparameters
lr = 2e-4
batch_size = 32
num_epochs = 20

training_args = TrainingArguments(
    output_dir=f"{MODEL_DIR}/scam",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_cpu=False
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Users\Alan\AppData\Local\Temp\ipykernel_21268\533835070.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.311900,0.324903,0.849000,0.939000
2,0.342700,0.350343,0.842000,0.940000
3,0.365400,0.306498,0.858000,0.944000
4,0.337200,0.337524,0.847000,0.946000
5,0.321300,0.310182,0.862000,0.947000


KeyboardInterrupt: 